In [ ]:
# Develop a program to classify the MNIST Dataset usingConvolution Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
xtrain = xtrain.reshape(xtrain.shape[0], 28, 28, 1).astype('float32') / 255.0
xtest = xtest.reshape(xtest.shape[0], 28, 28, 1).astype('float32') / 255.0
ytrain = to_categorical(ytrain, 10)
ytest = to_categorical(ytest, 10)
model = Sequential([
Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
MaxPooling2D(pool_size=(2, 2)),
Conv2D(64, kernel_size=(3, 3), activation='relu'),
MaxPooling2D(pool_size=(2, 2)),
Flatten(),
Dense(128, activation='relu'),
Dropout(0.5),
Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'])
history = model.fit(xtrain, ytrain,
epochs=10,
batch_size=64,
validation_data=(xtest, ytest))
testloss, testacc = model.evaluate(xtest, ytest)
print(f"Test Accuracy: {testacc:.2f}")
pred = model.predict(xtest[:5])
for i, prediction in enumerate(pred):
  print(f"Sample {i} - Predicted Label: {tf.argmax(prediction).numpy()}, True Label:{tf.argmax(ytest[i]).numpy()}")

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 55s 57ms/step - accuracy: 0.8348 - loss: 0.5240 - val_accuracy: 0.9843 - val_loss: 0.0511
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 52s 56ms/step - accuracy: 0.9699 - loss: 0.1034 - val_accuracy: 0.9868 - val_loss: 0.0399
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.9802 - loss: 0.0684 - val_accuracy: 0.9883 - val_loss: 0.0322
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 52s 55ms/step - accuracy: 0.9838 - loss: 0.0573 - val_accuracy: 0.9897 - val_loss: 0.0324
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 55ms/step - accuracy: 0.9869 - loss: 0.0447 - val_accuracy: 0.9910 - val_loss: 0.0261
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 80s 53ms/step - accuracy: 0.9877 - loss: 0.0396 - val_accuracy: 0.9925 - val_loss: 0.0264
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 84s 55ms/step - accuracy: 0.9897 - loss: 0.0328 - val_accuracy: 0.9925 - val_loss: 0.0242
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.9904 - loss: 0.0301 - 